In [ ]:
# President: 2016 (Trump), 2020 (Biden), 2024 (Trump)
# U.S. Senate: 2014 (Peters), 2018 (Stabenow), 2020 (Peters), 2024 (Slotkin)
# U.S. House: every cycle
# State Senate: 2014, 2018, 2022
# State House: every cycle

# THIS USES THE HOLDOUT PRED. WHICH CURRENTLY IS ONLY THIS:
OFFICES = ['U.S. House']
YEARS = ['2024']
# ALSO DON't FORGET BOTH PRED AND TRUE PLOTS - THERE IS A FLAG FOR THAT

# OFFICES = ['U.S. House', 'State House']
# YEARS = ['2018', '2020', '2022', '2024']

# OFFICES = ['U.S. Senate']
# YEARS = ['2014', '2018', '2020', '2024']

# OFFICES = ['State Senate']
# YEARS = ['2014', '2018', '2022']

# OFFICES = ['President']
# YEARS = ['2016', '2020', '2024']

In [ ]:
# TARGET = 'partisanship_lean_curr'
TARGET = 'partisan_temp'  # THIS IS THE HOLDOUT TARGET - USHOUSE 2024
# TARGET = 'predicted_partisan_temp'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

def makeSmoothPartisanColormap(n_colors=257):
    assert n_colors % 2 == 1, "Use an odd number for a center gray"
    half = n_colors // 2

    # Blue side for -1
    reds_neg = np.linspace(0, 127, half, dtype=int)
    greens_neg = np.linspace(0, 127, half, dtype=int)
    blues_neg = np.linspace(255, 127, half, dtype=int)
    blue_half = [(r, g, b) for r, g, b in zip(reds_neg, greens_neg, blues_neg)]

    # Center gray
    middle = [(128, 128, 128)]

    # Red side for +1
    reds_pos = np.linspace(129, 255, half, dtype=int)
    greens_pos = np.linspace(129, 0, half, dtype=int)
    blues_pos = np.linspace(129, 0, half, dtype=int)
    red_half = [(r, g, b) for r, g, b in zip(reds_pos, greens_pos, blues_pos)]

    full_rgb = blue_half + middle + red_half
    hex_colors = ['#{:02x}{:02x}{:02x}'.format(r, g, b) for r, g, b in full_rgb]
    return LinearSegmentedColormap.from_list("smooth_partisan", hex_colors, N=256)


from matplotlib.colors import TwoSlopeNorm

def get_centered_norm(df, column, soft_clip=0.3):
    return TwoSlopeNorm(vmin=-soft_clip, vcenter=0, vmax=soft_clip)

In [ ]:
# # FOR CATEGORICAL PREDS
# def makeColors():
#     n_colors = 17
#     half = n_colors // 2
    
#     # Blue gradient (Democratic, -1)
#     reds = np.linspace(0, 127, half, dtype=int)
#     greens = np.linspace(0, 127, half, dtype=int)
#     blues = np.linspace(255, 127, half, dtype=int)
#     first_half = [(r, g, b) for r, g, b in zip(reds, greens, blues)]

#     # Gray midpoint (neutral)
#     middle = [(128, 128, 128)]

#     # Red gradient (Republican, +1)
#     reds = np.linspace(129, 255, half, dtype=int)
#     greens = np.linspace(129, 0, half, dtype=int)
#     blues = np.linspace(129, 0, half, dtype=int)
#     second_half = [(r, g, b) for r, g, b in zip(reds, greens, blues)]

#     full_rgb = first_half + middle + second_half
#     hex_colors = ['#{:02x}{:02x}{:02x}'.format(r, g, b) for r, g, b in full_rgb]

#     return LinearSegmentedColormap.from_list("partisan_gradient", hex_colors, N=256)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import to_rgba

def plotPrecinctBounds(df, year, office, is_pred=True):
    
    prediction_column = TARGET

    fig, ax = plt.subplots(figsize=(80, 80))
    divider = make_axes_locatable(ax)

    df.boundary.plot(ax=ax, color="black", linewidth=0.1)
    
    # FOR CATEGORICAL PREDS
    cmap = makeSmoothPartisanColormap()
    
    # # FOR CATEGORICAL PREDS
    # colors = makeColors()
    
    norm = get_centered_norm(df, prediction_column, soft_clip=0.3)  # try 0.2 or 0.15 too

    # Normalize for color mapping
    norm = plt.Normalize(vmin=-1, vmax=1)
    df.plot(ax=ax, column=prediction_column, cmap=cmap, norm=norm, edgecolor="black", linewidth=0.01)

    ax.margins(0)
    ax.set_title('Precinct Preds', fontsize=64)
    ax.set_axis_off()

    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

    if is_pred:
        filename = f'output/maps/precincts/{year}_{office.replace(" ", "_").replace(".", "")}_Prediction_Map.png'
    else:
        filename = f'output/maps/precincts/{year}_{office.replace(" ", "_").replace(".", "")}_True_Map.png'
    
    plt.savefig(filename)
    # plt.show()
    plt.close(fig)

    # # FOR CATEGORICAL PREDS
    # colors = makeColors()
    # color_map = {
    #     "gargantuanly more republican": colors[0],
    #     "massively more republican": colors[1],
    #     "much much more republican": colors[2],
    #     "much more republican": colors[3],
    #     "more republican": colors[4],
    #     "slightly more republican": colors[5],
    #     "very slightly more republican": colors[6],
    #     "infinitesimally more republican": colors[7],
    #     "no change": colors[8],
    #     "infinitesimally more democrat": colors[9],
    #     "very slightly more democrat": colors[10],
    #     "slightly more democrat": colors[11],
    #     "more democrat": colors[12],
    #     "much more democrat": colors[13],
    #     "much much democrat": colors[14],
    #     "massively more democrat": colors[15],
    #     "gargantuanly more democrat": colors[16],
    #     "unknown": '#000000',
    # }
    # df['color'] = df[TARGET].apply(lambda x: color_map[x])
    # fig, ax = plt.subplots(figsize=(80, 80))
    # divider = make_axes_locatable(ax)
    # df.boundary.plot(ax=ax, color="black", linewidth=0.1)
    # df.plot(ax=ax, color=df['color'], edgecolor="black", linewidth=0.01)
    # ax.margins(0)
    # ax.set_title('Precinct Preds', fontsize=64)
    # ax.set_axis_off()
    # plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    # plt.savefig('output/maps/precincts/' + str(year) + "_" + office.replace(' ', '_').replace('.', '') + "_Prediction_Map.png")
    # plt.close(fig)

In [ ]:
for year in YEARS:
    for office in OFFICES:
        df_precinct_outcomes = gpd.read_file(f'data/generated_data/df_02_vote_changes_calc_{year}_{office.replace(' ', '_').replace('.', '')}.geojson', driver='GeoJSON')
        df_precinct_outcomes['standardized_id_num'] = df_precinct_outcomes['standardized_id_num'].astype(str).str.zfill(13)
        
        # df_precinct_pred_change_prev = pd.read_csv(f'data/generated_data/predicted_partisanship_change_prev_{year}_{office.replace(' ', '_').replace('.', '')}.csv')
        # df_precinct_pred_change_prev = pd.read_csv(f'data/generated_data/predicted_partisan_temp_{year}_{office.replace(' ', '_').replace('.', '')}.csv')
        df_precinct_pred_change_prev = pd.read_csv(f'data/generated_data/predicted_partisan_temp_{year}_{office.replace(' ', '_').replace('.', '')}.csv')
        df_precinct_pred_change_prev['standardized_id_num'] = df_precinct_pred_change_prev['standardized_id_num'].astype(int).astype(str).str.zfill(13)

        df_precinct_pred = df_precinct_outcomes.merge(df_precinct_pred_change_prev, on="standardized_id_num", how="inner")

        df_precinct_pred.drop(columns=['partisan_temp_y'], inplace=True)
        df_precinct_pred.rename(columns={'partisan_temp_x': 'partisan_temp'}, inplace=True)
        
        plotPrecinctBounds(df_precinct_pred, year, office, False)

print('DONE')